# Windrose

In [ ]:
import netCDF4

f = "wrfout_d01.nc"

outfilename = "wrfUV.nc"

in_nc = netCDF4(f, mode='r', format="NETCDF3_CLASSIC")
out_nc = netCDF4(outfilename, mode='w', format="NETCDF3_CLASSIC")

# Copy Global Attributes
for att_name in in_nc.ncattrs():
    setattr(out_nc, att_name, getattr(in_nc, att_name))

# Copy Dimensions, but modify the vertical dimensions
for dimname, dim in in_nc.dimensions.items():
    out_nc.createDimension(dimname, len(dim))
        
# Copy Variables and their Attributes, using the reduced vertical dimension
for varname, var in in_nc.variables.items():
    if varname in ("U", "V", "P", "XLAT", "XLONG", "XTIME"):
        datatype = var.datatype
        dimensions = var.dimensions
        shape = var.shape
        
        new_shape = shape
        
        new_var = out_nc.createVariable(varname, datatype, dimensions)
        
        new_var[:] = var[:]
        
        for att_name in var.ncattrs():
            setattr(new_var, att_name, getattr(var, att_name))
            
in_nc.close()
out_nc.close()

In [4]:
from __future__ import print_function
import netCDF4
import wrf
import windrose
import numpy
import matplotlib
%matplotlib inline
import matplotlib.animation
import ipywidgets
import IPython.display 

ncfile = netCDF4.Dataset("wrfUV.nc")

In [5]:
WSPD, WDIR = wrf.g_wind.get_destag_wspd_wdir(ncfile,units='m s-1')
wspd = wrf.to_np(WSPD)
wdir = wrf.to_np(WDIR)

array([[[  7.03830051,   6.06134748,   5.2927494 , ...,   7.16538954,
           6.89305258,   7.01743603],
        [  8.86670589,   6.41850281,   5.71640015, ...,   4.11433458,
           2.89117074,   5.4950161 ],
        [ 10.57899189,   6.84130955,   5.02981949, ...,   1.9991678 ,
           2.33127093,   5.13581133],
        ..., 
        [  5.03965664,   6.43416357,   6.09475422, ...,   2.10683084,
           1.89265466,   1.4026171 ],
        [  5.08885717,   6.06781435,   5.41861629, ...,   1.97418952,
           2.01908302,   0.9927786 ],
        [  5.1214819 ,   5.51015615,   5.07224035, ...,   1.53556156,
           1.57553911,   1.97280443]],

       [[  9.00567055,   7.93283653,   7.48407078, ...,   9.09790897,
           9.01935387,   9.3504715 ],
        [ 10.47212315,   8.15084934,   7.43056059, ...,   4.91137409,
           4.83009672,   7.49126434],
        [ 12.17402554,   8.90717983,   7.05489492, ...,   0.63702452,
           3.53737998,   6.94883156],
        ...,

In [9]:
def handle_slider_change(height, *args):
    windrose.plot_windrose_np(wdir[height], var= wspd[height], 
                           kind='contourf', lw=6, nsector=16, bins = 6)
    
interactive_plot = ipywidgets.interactive(handle_slider_change, height= (0,wspd.shape[0]-1, 1))
output = interactive_plot.children[-1]
output.layout.height = '640px'   #set the height of the interactive plot so it won't change size
interactive_plot

interactive(children=(IntSlider(value=29, description='height', max=58), Output(layout=Layout(height='640px'))), _dom_classes=('widget-interact',))

In [12]:
button_save = ipywidgets.Button(description='save windrose')
display(button_save)
def on_button_clicked(b):
    matplotlib.pyplot.savefig('windrose_h'+str(interactive_plot.children[0].value).zfill(len(str(wspd.shape[0])))+
                              '.png')

button_save.on_click(on_button_clicked)
def handle_slider_change(height, *args):
    windrose.plot_windrose_np(wdir[height], var= wspd[height], 
                           kind='contourf', lw=6, nsector=16, bins = 6)

interactive_plot = ipywidgets.interactive(handle_slider_change, height= (0,wdir.shape[0]-1, 1))
output = interactive_plot.children[-1]
output.layout.height = '640px'  
interactive_plot

Button(description='save windrose', style=ButtonStyle())

interactive(children=(IntSlider(value=29, description='height', max=58), Output(layout=Layout(height='640px'))), _dom_classes=('widget-interact',))

In [ ]:
def drawwindrose(h,ax, title):
    ax.set_title(title, fontsize=14)
    cs = ax.contourf(wdir[h], wspd[h], nsector=16,lw=6)
    ax.set_legend()
    return cs
def myanimate(h):   
    ax = windrose.WindroseAxes.from_ax(fig=fig) 
    new_contour = drawwindrose(h,ax, 'Wind Speed Distribution: Frame %03d'%(h)) 
    return new_contour
FFMpegWriter = matplotlib.animation.writers['ffmpeg']
metadata = dict(title='Wind Speed Distribution from 2018-01-09T06:00:00', artist='Metos@UIO',
                comment='Movie for Wind Speed Distribution from 2018-01-09T06:00:00')
writer = FFMpegWriter(fps=20, metadata=metadata, bitrate=1800)

fig= matplotlib.pyplot.figure(figsize=[10,10])

ani = matplotlib.animation.FuncAnimation(fig, myanimate, frames=numpy.arange(wdir.shape[0]), 
                                         interval=50*wdir.shape[0])
ani.save("writer_WindSpeedDistribution_200180109.mp4", writer = writer)

In [ ]:
IPython.display.HTML(ani.to_html5_video())

In [ ]:
def handle_slider_change(height, *args):
    ax = windrose.WindroseAxes.from_ax()
    ax.bar(wdir[height],var=wspd[height],nsector=16,bins=6,
           normed=True, opening=0.8, edgecolor='white')
    ax.set_legend()
    ax.set_title('Wind Speed Distribution: Frame %03d'%(height),fontsize=14)
    #windrose.plot_windrose_np(numpy.rad2deg(direction[height]),var=speed[height],kind='contourf',lw=6,nsector=16,bins=6)
interactive_plot = ipywidgets.interactive(handle_slider_change, height= (0, wspd.shape[0]-1, 1))
output = interactive_plot.children[-1]
output.layout.height = '640px'   #set the height of the interactive plot so it won't change size
interactive_plot

In [ ]:
button_save = ipywidgets.Button(description='save windrose')
display(button_save)
def on_button_clicked(b):
    matplotlib.pyplot.savefig('windrose_h'+str(interactive_plot.children[0].value).zfill(len(str(wspd.shape[0])))+
                              '.png')

button_save.on_click(on_button_clicked)
def handle_slider_change(height, *args):
    ax = windrose.WindroseAxes.from_ax()
    ax.bar(wdir[height],var=wspd[height],nsector=16,bins=6,
           normed=True, opening=0.8, edgecolor='white')
    ax.set_legend()
    ax.set_title('Wind Speed Distribution: Frame %03d'%(height),fontsize=14)

interactive_plot = ipywidgets.interactive(handle_slider_change, height= (0,wspd.shape[0]-1, 1))
output = interactive_plot.children[-1]
output.layout.height = '640px'  
interactive_plot

In [ ]:
def drawwindrose(h,ax, title):
    ax.set_title(title, fontsize=14)
    cs = ax.bar(wdir[h],var=wspd[h],nsector=16,bins=6,normed=True, opening=0.8, edgecolor='white')
    ax.set_legend()
    return cs
def myanimate(h):   
    ax = windrose.WindroseAxes.from_ax(fig=fig) 
    new_contour = drawwindrose(h,ax, 'Wind Speed Distribution: Frame %03d'%(h)) 
    return new_contour
FFMpegWriter = matplotlib.animation.writers['ffmpeg']
metadata = dict(title='Wind Speed Distribution from 2018-01-09T06:00:00', artist='Metos@UIO',
                comment='Movie for Wind Speed Distribution from 2018-01-09T06:00:00')
writer = FFMpegWriter(fps=20, metadata=metadata, bitrate=1800)

fig= matplotlib.pyplot.figure(figsize=[10,10])

ani2 = matplotlib.animation.FuncAnimation(fig, myanimate, frames=numpy.arange(wdir.shape[0]), 
                                         interval=50*wdir.shape[0])
ani2.save("writer_WindSpeedDistributionnormed_200180109.mp4", writer = writer)

In [ ]:
IPython.display.HTML(ani2.to_html5_video())

In [ ]:
def css_styling(filepath):
    styles = open(filepath, 'r').read()
    return HTML(styles)
css_styling('/styles/custom.css')